## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP1 - Preparación de los datos

---

En esta libreta se realizan las transforamciones sobre los datasets del escenario en función 
de los resultados de la verificación de calidad de datos. 

---

In [138]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

In [139]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_mc.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_mc.csv", sep=";")
display(df_tarjetas.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0


---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos
4. Integración de datasets
5. Formateo definitivo


----

Selección de datos

In [140]:
# Se establece qué columnas se eliminan

col_eliminar_creditos = ['antiguedad_empleado']
col_eliminar_tarjetas = ['limite_credito_tc', 'genero', 'personas_a_cargo', 'nivel_educativo', 'operaciones_ult_12m']

# Se ejecuta la operación

df_creditos.drop(col_eliminar_creditos, inplace=True, axis=1)
df_tarjetas.drop(col_eliminar_tarjetas, inplace=True, axis=1)

In [141]:
print("Vista del dataset de datos de créditos:")
display(df_creditos.head(1))

print("Vista del dataset de datos de tarjetas:")
display(df_tarjetas.head(1))

Vista del dataset de datos de créditos:


,id_cliente,edad,importe_solicitado,duracion_credito,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


Vista del dataset de datos de tarjetas:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,nivel_tarjeta
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,Blue


Limpieza de los datos (filtrado a nivel de filas)

Mi primer paso es filtrar las filas según dicta la primera regla de negocio

In [142]:
#Regla 1
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_creditos.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_pct_ingresos], axis=1)
df_creditos.head(5)  

,id_cliente,edad,importe_solicitado,duracion_credito,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,regla_pct_ingresos
0,713061558.0,22,35000,3,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y,ok
1,768805383.0,21,1000,2,PROPIA,9600,EDUCACIÓN,0.10,11.14,0,N,ok
2,818770008.0,25,5500,3,HIPOTECA,9600,SALUD,0.57,12.87,1,N,err
3,713982108.0,23,35000,2,ALQUILER,65500,SALUD,0.53,15.23,1,N,ok
4,710821833.0,24,35000,4,ALQUILER,54400,SALUD,0.55,14.27,1,Y,ok


A continuación procedo a filtrar según la segunda regla de negocio

In [143]:
# Regla 2
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito_minimo_propiedad(row, min_valor):
    pct_ingreso = row.pct_ingreso
    tipo_propietario = row.situacion_vivienda
    dur_credito = row.duracion_credito
    
    if pct_ingreso > 0.6 and tipo_propietario != 'PROPIA' and min_valor == dur_credito:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'
    
# Como en la documentación no indica nada al respecto de cuanto es el mínimo permitido de duración de un crédito, tomaré el valor mínimo de la columna
min_valor = df_creditos['duracion_credito'].min()

# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_creditos.apply(lambda row: regla_pct_ingresos_credito_minimo_propiedad(row, min_valor), axis=1).rename("regla_pct_ingresos_credito_minimo_propiedad")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_pct_ingresos], axis=1)
df_creditos.head(5)

,id_cliente,edad,importe_solicitado,duracion_credito,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,regla_pct_ingresos,regla_pct_ingresos_credito_minimo_propiedad
0,713061558.0,22,35000,3,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y,ok,ok
1,768805383.0,21,1000,2,PROPIA,9600,EDUCACIÓN,0.10,11.14,0,N,ok,ok
2,818770008.0,25,5500,3,HIPOTECA,9600,SALUD,0.57,12.87,1,N,err,ok
3,713982108.0,23,35000,2,ALQUILER,65500,SALUD,0.53,15.23,1,N,ok,ok
4,710821833.0,24,35000,4,ALQUILER,54400,SALUD,0.55,14.27,1,Y,ok,ok


In [144]:
# Se filtran las filas con algún error detectado
print(f"Filas antes del filtro: {df_creditos.shape[0]}")

# Filtro descartando edades mayores a sesenta años
temp = df_creditos[df_creditos['edad'] < 90]

# Filtro descartando porcentajes mayores al 70 por cioento ya que sería irreal en un  caso real bancario
temp_a = temp[temp['pct_ingreso'] < 0.7]

# Otro filtro posible: por la regla de negocio agregada
temp_b = temp_a[temp_a['regla_pct_ingresos'] == 'ok']

#Otro filtro posible: por la segunda regla de negocio agregada
temp_c = temp_b[temp_b['regla_pct_ingresos_credito_minimo_propiedad'] == 'ok']

print(f"Filas después del filtro: {temp_c.shape[0]}")

Filas antes del filtro: 10127
Filas después del filtro: 10102


Procedo a eliminar los valores nulos del atributo tasa_interes asignando el valor medio de la columna ya que me parece lo más coherente
He decidido no eliminar dicho atributo ya que considero que el interés de un crédito es un factor fundamental a la hora de solicitar uno.

In [145]:
# Primero calculo el valor medio del atributo
val_med = temp_c['tasa_interes'].mean()

# Posteriorment asigno el valor medio del atribtuo a los nulos
temp_c['tasa_interes'].fillna(val_med, inplace=True)

# Verfico que he eliminado los valores nulos
nulls = temp_c['tasa_interes'].isnull().sum()

if nulls == 0:
    print("Se han substituido todos los valores nulos del atributo tasa_interes por la media correspondiente.")

Se han substituido todos los valores nulos del atributo tasa_interes por la media correspondiente.


C:\Users\victo\AppData\Local\Temp\ipykernel_14504\435584869.py:5: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\victo\AppData\Local\Temp\ipykernel_14504\435584869.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Integración de datos

In [146]:
# Se integra el dataset por el atributo "id_cliente"
df_integrado = pd.merge(temp_c, df_tarjetas, on='id_cliente', how='inner')
coincidencias = df_integrado.shape[0]

print(f"Filas del dataset integrado con los filtros realizados: {coincidencias}")

Filas del dataset integrado con los filtros realizados: 10102


In [147]:
# Tener en cuenta que se han agregado dos columnas nuevas al aplicar las reglas de negocio para poder evaluarlas. 
# Destacar que estas columnas podrán ser eliminadas posteriormente.
print(f"Cantidad de columnas del dataset integrado: {df_integrado.shape[1]}")

Cantidad de columnas del dataset integrado: 18


#### Transformación de atributos

Atributos nominales que se modifican los valores

In [148]:
# Columna: situacion_vivienda
cambios_situacion_vivienda = {
    'ALQUILER' : 'A',
    'PROPIA' : 'P',
    'HIPOTECA' : 'H',
    'OTROS' : 'O',
}

situacion_vivienda_N = df_integrado.loc[:, ('situacion_vivienda')].map(cambios_situacion_vivienda).rename('situacion_vivienda_N')

# Columna: objetivo_credito
cambios_objetivo_credito = {
    'PERSONAL' : 'P',
    'EDUCACIÓN' : 'E',
    'SALUD' : 'S',
    'INVERSIONES' : 'I',
    'MEJORAS_HOGAR' : 'MH',
    'PAGO_DEUDAS' : 'PD',
}

objetivo_credito_N = df_integrado.loc[:, ('objetivo_credito')].map(cambios_objetivo_credito).rename('objetivo_credito_N')

# Columna: falta_pago
cambios_falta_pago = {
    'Y' : 'S',
    'N' : 'N',
}

falta_pago_N = df_integrado.loc[:, ('falta_pago')].map(cambios_falta_pago).rename('falta_pago_N')

# Columna: estado_civil
cambios_estado_civil = {
    'CASADO' : 'C',
    'SOLTERO' : 'S',
    'DESCONOCIDO' : 'N',
    'DIVORCIADO' : 'D',
}

estado_civil_N = df_integrado.loc[:, ('estado_civil')].map(cambios_estado_civil).rename('estado_civil_N')

# Columna: estado_cliente
cambios_estado_cliente = {
    'ACTIVO' : 'A',
    'PASIVO' : 'P',
}

estado_cliente_N = df_integrado.loc[:, ('estado_cliente')].map(cambios_estado_cliente).rename('estado_cliente_N')

# Columna: nivel_tarjeta
cambios_nivel_tarjeta = {
    'Blue' : 'B',
    'Gold' : 'G',
    'Silver' : 'S',
    'Platinum' : 'P',
}

nivel_tarjeta_N = df_integrado.loc[:, ('nivel_tarjeta')].map(cambios_nivel_tarjeta).rename('nivel_tarjeta_N')

# Columna: estado_credito
cambios_estado_credito = {
    0: 'P',
    1 : 'C',
}

estado_credito_N = df_integrado.loc[:, ('estado_credito')].map(cambios_estado_credito).rename('estado_credito_N')

# Lo realizaré al final con las numéricas, obteniendo el df_final (teniendo en cuenta tanto atributos nominales como numéricos)
# df_final = pd.concat([situacion_vivienda_N, objetivo_credito_N, falta_pago_N, estado_civil_N, estado_cliente_N, nivel_tarjeta_N, estado_credito_N, df_integrado], axis=1)
# df_final.head()

# Sobre este resultado será necesario eliminar las columnas auxiliares

Atributos numéricos que se discretizan

In [149]:
df_integrado['edad'].describe()

count    10102.000000
mean        23.529895
std          1.510367
min         20.000000
25%         22.000000
50%         23.000000
75%         25.000000
max         26.000000
Name: edad, dtype: float64

In [150]:
# edad

etiquetas_e = ['menor_25', '25_a_30']
rangos_e = [0, 24, 50]
edad_N = pd.cut(df_integrado['edad'], 
                                bins=rangos_e, 
                                labels=etiquetas_e)

edad_N.value_counts()

# pct_ingreso

etiquetas_p_i = ['hasta_20', '20_a_40', '40_a_60', 'mayor_60']
rangos_p_i = [0, 0.19, 0.39, 0.60, 0.99]
pct_ingreso_N = pd.cut(df_integrado['pct_ingreso'], 
                                bins=rangos_p_i, 
                                labels=etiquetas_p_i)

pct_ingreso_N.value_counts()

# ingresos

etiquetas_i = ['hasta_20k', '20k_a_50k', '50k_a_100k', 'mayor_100k']
rangos_i = [0, 19999, 49999, 99999, 999999]
ingresos_N = pd.cut(df_integrado['ingresos'], 
                                bins=rangos_i, 
                                labels=etiquetas_i)

ingresos_N.value_counts()

# tasa_interes

etiquetas_t_i = ['hasta_7p', '7p_a_15p', '15p_a_20p', 'mayor_20p']
rangos_t_i = [0, 6.99, 14.99, 19.99, 100]
tasa_interes_N = pd.cut(df_integrado['tasa_interes'], 
                                bins=rangos_t_i, 
                                labels=etiquetas_t_i)

tasa_interes_N.value_counts()

# antiguedad_cliente

etiquetas_a_c = ['menor_2y', '2y_a_4y', 'mayor_4y']
rangos_a_c = [0, 24, 48, 100]
antiguedad_cliente_N = pd.cut(df_integrado['antiguedad_cliente'], 
                                bins=rangos_a_c, 
                                labels=etiquetas_a_c)

antiguedad_cliente_N.value_counts()

# gastos_ult_12m

etiquetas_g_u12 = ['menor_1k', '1k_a_4k', '4k_a_6k', '6k_a_8k', '8k_a_10k', 'mayor_10k']
rangos_g_u12 = [0, 999, 3999, 5999, 7999, 9999, 100000]
gastos_ult_12m_N = pd.cut(df_integrado['gastos_ult_12m'], 
                                bins=rangos_g_u12, 
                                labels=etiquetas_g_u12)

gastos_ult_12m_N.value_counts()

gastos_ult_12m
1k_a_4k      5107
4k_a_6k      3386
mayor_10k     757
6k_a_8k       361
8k_a_10k      331
menor_1k      160
Name: count, dtype: int64

In [151]:
df_integrado.columns

Index(['id_cliente', 'edad', 'importe_solicitado', 'duracion_credito',
       'situacion_vivienda', 'ingresos', 'objetivo_credito', 'pct_ingreso',
       'tasa_interes', 'estado_credito', 'falta_pago', 'regla_pct_ingresos',
       'regla_pct_ingresos_credito_minimo_propiedad', 'antiguedad_cliente',
       'estado_civil', 'estado_cliente', 'gastos_ult_12m', 'nivel_tarjeta'],
      dtype='object')

In [152]:
# Elimino las columnas que numéricas no normalizads y también aprovecho para eliminar las columnas de evaluación de las reglas de negocio.
col_eliminar_final_numericas = [
              'edad',
              'antiguedad_cliente', 
              'ingresos',
              'pct_ingreso', 
              'tasa_interes',
              'regla_pct_ingresos',
              'regla_pct_ingresos_credito_minimo_propiedad',
              'gastos_ult_12m',
              'id_cliente']

# Elimino las columnas nominales ya que utilizaré las columna con la tranformación aplicada que permite una mejor lectura de los datos.
col_eliminar_final_nominales = [
              'situacion_vivienda',
              'objetivo_credito', 
              'falta_pago',
              'estado_civil', 
              'estado_cliente',
              'nivel_tarjeta',
              'estado_credito']

df_integrado.drop(col_eliminar_final_numericas, inplace=True, axis=1)
df_integrado.drop(col_eliminar_final_nominales, inplace=True, axis=1)


df_final = pd.concat([gastos_ult_12m_N, antiguedad_cliente_N, tasa_interes_N, ingresos_N, pct_ingreso_N, edad_N, situacion_vivienda_N, objetivo_credito_N, falta_pago_N, estado_civil_N, estado_cliente_N, nivel_tarjeta_N, estado_credito_N, df_integrado], axis=1)
df_final.head(5)


,gastos_ult_12m,antiguedad_cliente,tasa_interes,ingresos,pct_ingreso,edad,situacion_vivienda_N,objetivo_credito_N,falta_pago_N,estado_civil_N,estado_cliente_N,nivel_tarjeta_N,estado_credito_N,importe_solicitado,duracion_credito
0,1k_a_4k,2y_a_4y,15p_a_20p,50k_a_100k,40_a_60,menor_25,A,P,S,C,A,B,C,35000,3
1,1k_a_4k,2y_a_4y,7p_a_15p,hasta_20k,hasta_20,menor_25,P,E,N,C,A,B,P,1000,2
2,1k_a_4k,2y_a_4y,15p_a_20p,50k_a_100k,40_a_60,menor_25,A,S,N,C,A,B,C,35000,2
3,1k_a_4k,mayor_4y,7p_a_15p,50k_a_100k,40_a_60,menor_25,A,S,S,C,A,B,C,35000,4
4,1k_a_4k,2y_a_4y,7p_a_15p,hasta_20k,20_a_40,menor_25,P,I,N,N,A,B,C,2500,2


Exportación de metadatos

In [153]:
import dtale as dt

informe = dt.show(df_final)
informe.open_browser()

In [154]:
print(f"Cantidad de columnas del dataset final: {df_final.shape[1]}")
print(f"Cantidad de filas del dataset final: {df_final.shape[0]}")

Cantidad de columnas del dataset final: 15
Cantidad de filas del dataset final: 10102


Exportación del dataset

In [155]:
df_final.to_csv("../../data/final/datos_finales.csv", sep=';', index=False)